In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Dataset - https://www.kaggle.com/datasets/deadskull7/fer2013

df = pd.read_csv('/content/fer2013.csv')

In [ ]:
df.head(10)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
5,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
6,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
7,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
8,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training
9,2,255 254 255 254 254 179 122 107 95 124 149 150...,Training


In [ ]:
df.tail(10)

,emotion,pixels,Usage
35877,6,139 143 145 154 159 168 176 181 190 191 195 19...,PrivateTest
35878,3,0 39 81 80 104 97 51 64 68 46 41 67 53 68 70 5...,PrivateTest
35879,2,0 0 6 16 19 31 47 18 26 19 17 8 15 3 4 2 14 20...,PrivateTest
35880,2,164 172 175 171 172 173 178 181 188 192 197 20...,PrivateTest
35881,0,181 177 176 156 178 144 136 132 122 107 131 16...,PrivateTest
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest
35886,2,19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...,PrivateTest


In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
emotion,35887.0,3.323265,1.873819,0.0,2.0,3.0,5.0,6.0


In [ ]:
def pixels_into_array(pixel_string):
  pixel_array = np.fromstring(pixel_string, dtype=int, sep=' ')
  return pixel_array.reshape(48, 48)

In [ ]:
df['emotion'].unique()

array([0, 2, 4, 6, 3, 5, 1])

In [ ]:
X = df['pixels'].apply(pixels_into_array)
y = df['emotion']

In [ ]:
X = np.stack(X)
X = X / 255.0
X = X.reshape(-1, 48, 48, 1)
y = to_categorical(y, num_classes=7)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(7, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
data_aug = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.1)

In [ ]:
train_data = data_aug.flow(X_train, y_train, batch_size = 32)

In [ ]:
model.fit(train_data, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 695s 772ms/step - accuracy: 0.2282 - loss: 1.8324 - val_accuracy: 0.2498 - val_loss: 1.7823
Epoch 2/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 741s 771ms/step - accuracy: 0.2535 - loss: 1.7898 - val_accuracy: 0.2914 - val_loss: 1.7072
Epoch 3/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 741s 770ms/step - accuracy: 0.2824 - loss: 1.7424 - val_accuracy: 0.3176 - val_loss: 1.6820
Epoch 4/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 741s 770ms/step - accuracy: 0.3086 - loss: 1.6884 - val_accuracy: 0.4216 - val_loss: 1.4976
Epoch 5/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 701s 780ms/step - accuracy: 0.3650 - loss: 1.6008 - val_accuracy: 0.4645 - val_loss: 1.4087
Epoch 6/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 698s 777ms/step - accuracy: 0.3987 - loss: 1.5396 - val_accuracy: 0.4667 - val_loss: 1.4003
Epoch 7/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 702s 781ms/step - accuracy: 0.4180 - loss: 1.4884 - val_accuracy: 0.4887 - val_loss: 1.3413
Epoch 8/20
898/898 ━━━━━━━━━━━━━━━━━━━━ 775s 819ms/step - accuracy: 0.4335 -

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

225/225 ━━━━━━━━━━━━━━━━━━━━ 48s 215ms/step - accuracy: 0.5372 - loss: 1.2061
Test Loss: 1.1831847429275513
Test Accuracy: 0.5493173599243164


In [ ]:
y_pred = model.predict(X_test)

225/225 ━━━━━━━━━━━━━━━━━━━━ 49s 217ms/step


In [ ]:
y_pred_labels = np.argmax(y_pred, axis=1)
print('Accuracy Score:', accuracy_score(y_test.argmax(axis=1), y_pred_labels))

Accuracy Score: 0.5493173585957091
